In [1]:
import os
os.environ["TRANSFORMERS_OFFLINE"] = "1"  # <- ini penting

from unsloth import FastLanguageModel
model_path = r"C:\Users\Kentdry\Documents\VSCODE\TA1(Deepseek)\models--unsloth--llama-3.2-1b-instruct-unsloth-bnb-4bit\snapshots\0a4436e20494a6504464ce35274b7e53fb7883d0"  # lengkapin path-nya
max_seq_length = 2048  # Maximum number of tokens processed at once
dtype = None  # Default data type (adjusts automatically)
load_in_4bit = True  # Enable 4-bit quantization to save memory
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


c:\Users\Kentdry\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


c:\Users\Kentdry\AppData\Local\Programs\Python\Python310\lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 3050 6GB Laptop GPU. Num GPUs = 1. Max memory: 6.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv("Gabungan_data_TA1.csv", na_values=["#N/A", "#n/a", "n/a", "NA", "na"])
important_columns = [
    "sem_01_GPA", "sem_02_GPA", "sem_03_GPA",
    "sem_01_CGPA", "sem_02_CGPA", "sem_03_CGPA",
    "application_count", "major_name_opcs", 
    "gender"
]
df_cleaned = df.dropna(subset=important_columns)
df_cleaned = df_cleaned.drop(columns=["NIM", "sem_01_CGPA","sem_02_CGPA","form_number","application_count","school_geo_unit","sem_01_GPA", "sem_02_GPA", "sem_03_GPA"])
dataset_ds = Dataset.from_pandas(df_cleaned)


split_dataset = dataset_ds.train_test_split(test_size=0.05, seed=42)
dataset_ds_train = split_dataset["train"]
dataset_ds_test = split_dataset["test"]

dataset_ds_train = dataset_ds_train.to_pandas()
dataset_ds_train.to_csv("dataset_ds_train.csv", index=False)


In [3]:
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset_ds_test ,
    merged_prompt = (
        "The following are the student High School grade and student data."
        "[[The student scored {ENG} in English, and {MATH} in Math.]]"
        "[[ They also scored {BIO} in Biology,{CHEM} in Chemistry, and {PHY} in Physics.]]"
        "[[ They also scored {ECON} in Economics,{GEO} in Geography, and {SOC} in Social.]]"
        "[[ They scored {FINAL} on their final year of high school exam.]]"
        "[[ They applied {major_name_opcs} as a major.]]"
        "[[ Their father's occupation is {father_occupation} and mother's occupation is {mother_occupation}.]]"
        "[[ The student is {gender} and studied at {school_name} in {school_state}.]]"
        "[[ The student takes the {curriculum_name} curriculum and is classified as {school_prop} school.]]"
    ),
    conversation_extension = 1,
    output_column_name = "sem_03_CGPA",
)

Converting to ShareGPT: 100%|██████████| 118/118 [00:00<00:00, 39261.29 examples/s]


In [4]:
def convert_all_to_alpaca_format(dataset):
    alpaca_data = []
    conversations = dataset['conversations']
    instruction = "Based on the data given, predict their Cumulative GPA for the third semester"

    for i in range(len(conversations)):
        if conversations[i][0]['from'] == 'human' and conversations[i][1]['from'] == 'gpt':
            alpaca_data.append({
                "instruction": instruction,
                "input": conversations[i][0]['value'],
                "output": "their third semester cumulative GPA is " + conversations[i][1]['value']
            })

    return alpaca_data

alpaca_dataset = convert_all_to_alpaca_format(dataset)



In [5]:
# We need to format the dataset to fit our prompt training style 
EOS_TOKEN = tokenizer.eos_token  # Define EOS_TOKEN which the model when to stop generating text during training
EOS_TOKEN

'<|eot_id|>'

In [6]:
chat_templates="""Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question 
    ### Instruction:
    {}
    ### Input:
    {}
    ### Response:
    {}"""

def formatting_prompts_func(example):
    instructions = example["instruction"]
    inputs       = example["input"]
    outputs      = example["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = chat_templates.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import Dataset
# Konversi list of dict ke HuggingFace Dataset
alpaca_dataset = Dataset.from_list(alpaca_dataset)
alpaca_dataset = alpaca_dataset.map(formatting_prompts_func, batched = True)

alpaca_dataset.to_json("dataset_ds_test.json", orient="records", lines=True)


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 475.28ba/s]


186282